In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls

Mounted at /content/gdrive
gdrive	sample_data


In [3]:
%cd gdrive/My\ Drive/work/GB_NLP/6

/content/gdrive/My Drive/work/GB_NLP/6


In [ ]:
!ls

addition_rnn.ipynb			    summarization.ipynb
classic_poems.json			    summarization_new.ipynb
evgenyi_onegin.txt			    summ_attention.ipynb
image_captioning.ipynb			    text_classification_rnn.ipynb
lstm_character_level_text_generation.ipynb  text_generation.ipynb
nmt.ipynb				    text_gen.ipynb
nmt_with_attention.ipynb		    training_checkpoints
rnn.ipynb				    transformer_actual.ipynb


In [4]:
path_to_file = 'evgenyi_onegin.txt'

In [5]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 286984 characters


In [6]:
print(text[:500])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                        Хотел бы я тебе представить
                        Залог достойнее тебя,
                        Достойнее души прекрасной,
                        Святой исполненной мечты,
                        Поэзии живой и ясной,
                        Высо


In [7]:
text = text + text

In [9]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

131 unique characters


In [10]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [11]:
text_as_int, text, len(text_as_int), len(text)

(array([ 71, 110, 104, ..., 104, 121,   0]),
 'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                                Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                        Вниманье дружбы возлюбя,\n                        Хотел бы я тебе представить\n                        Залог достойнее тебя,\n                        Достойнее души прекрасной,\n                        Святой исполненной мечты,\n                        Поэзии живой и ясной,\n                        Высоких дум и простоты;\n                        Но так и быть - рукой пристрастной\n                        Прими собранье пестрых глав,\n                        Полусмешных, полупечальных,\n                        Простонародных, идеальных,\n                        Небрежный плод моих забав,\n                        Бессонниц, легких вдохновений,\n                        Незрелых и увядших лет,\n                        Ума холодных наблюде

### train and target

In [12]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

А
л
е
к
с


In [13]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '
'      Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                        '
'Вниманье дружбы возлюбя,\n                        Хотел бы я тебе представить\n                        '
'Залог достойнее тебя,\n                        Достойнее души прекрасной,\n                        Свят'
'ой исполненной мечты,\n                        Поэзии живой и ясной,\n                        Высоких д'


In [14]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first example input and target values:

In [15]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                         '
Target data: 'лександр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '


In [16]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [18]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 1024

In [ ]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True),
        tf.keras.layers.Dense(vocab_size)
    ]
)

In [ ]:
class RNNgenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, batch_size):
        super(RNNgenerator, self).__init__()
        
        self.emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)
                                 
        self.gru1 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
        self.gru2 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
                           
        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        emb_x = self.emb(x)
        x1 = self.gru1(emb_x)
        x = x1
        for _ in range(3):
            x = self.gru2(x)
        #x = self.gru1(x)
        x = (x + x1)/2
        return self.fc(x)

model = RNNgenerator(vocab_size, embedding_dim, BATCH_SIZE)

In [19]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
                                 
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [20]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [21]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 131) # (batch_size, sequence_length, vocab_size)


In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 128)           16768     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          4722688   
_________________________________________________________________
lstm_1 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
lstm_2 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
lstm_3 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
dense (Dense)                (64, None, 131)           134275    
Total params: 30,051,843
Trainable params: 30,051,843
Non-trainable params: 0
____________________________________________

In [ ]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 131), dtype=float32, numpy=
array([[ 2.0969965e-06,  8.7369399e-06,  1.4929498e-05, ...,
        -1.1595096e-05, -5.7436928e-06, -1.3965824e-05],
       [ 7.9077263e-06,  3.3559423e-05,  5.7170491e-05, ...,
        -4.7259757e-05, -2.8301025e-05, -5.7697434e-05],
       [ 1.7341581e-05,  7.6751021e-05,  1.3195755e-04, ...,
        -1.1546607e-04, -8.0640930e-05, -1.4257040e-04],
       ...,
       [-1.4742030e-03,  6.9336726e-05,  4.4791558e-04, ...,
         5.5562111e-04, -3.1631480e-03,  4.2857294e-04],
       [-1.4436652e-03,  1.3969278e-04,  4.9531541e-04, ...,
         4.7246186e-04, -2.9527938e-03,  4.1066157e-04],
       [-1.4286742e-03,  1.5175615e-04,  5.3618004e-04, ...,
         3.9178820e-04, -2.7288469e-03,  3.9515935e-04]], dtype=float32)>

In [23]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [24]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'ли все.\n                        И бал блестит во всей красе.\n\n                                      '

Next Char Predictions: 
 '!XA8ыmf;ЬcNЭ8)паП\'с;;\'qzчОУаП?СTnscтVьnGф.Дх"эФ2чnFcКs:1HШб)!шпГБАbl}юЬФФXyemнs"d-PvFйУнббfПШoHVFMРд'


## Train the model

In [25]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 131)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.8752046


In [26]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [ ]:
!rm -rf ./training_checkpoints

In [ ]:
!ls ./training_checkpoints

ls: cannot access './training_checkpoints': No such file or directory


In [28]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*3,
    save_weights_only=True)

### Execute the training

In [31]:
EPOCHS = 20

In [32]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
88/88 [==============================] - 30s 330ms/step - loss: 1.5065
Epoch 2/20
88/88 [==============================] - 32s 361ms/step - loss: 1.3854
Epoch 3/20
88/88 [==============================] - 30s 336ms/step - loss: 1.3452
Epoch 4/20
88/88 [==============================] - 30s 340ms/step - loss: 1.3117
Epoch 5/20
88/88 [==============================] - 32s 356ms/step - loss: 1.2734
Epoch 6/20
88/88 [==============================] - 31s 340ms/step - loss: 1.2346
Epoch 7/20
88/88 [==============================] - 31s 344ms/step - loss: 1.2013
Epoch 8/20
88/88 [==============================] - 32s 359ms/step - loss: 1.1917
Epoch 9/20
88/88 [==============================] - 31s 343ms/step - loss: 1.1465
Epoch 10/20
88/88 [==============================] - 31s 345ms/step - loss: 1.1565
Epoch 11/20
88/88 [==============================] - 33s 368ms/step - loss: 1.0924
Epoch 12/20
88/88 [==============================] - 31s 344ms/step - loss: 1.0387
Epoch 13/20
8

## Generate text

In [35]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_20'

In [36]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [37]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 128)            16768     
_________________________________________________________________
lstm_4 (LSTM)                (1, None, 1024)           4722688   
_________________________________________________________________
lstm_5 (LSTM)                (1, None, 1024)           8392704   
_________________________________________________________________
lstm_6 (LSTM)                (1, None, 1024)           8392704   
_________________________________________________________________
lstm_7 (LSTM)                (1, None, 1024)           8392704   
_________________________________________________________________
dense_1 (Dense)              (1, None, 131)            134275    
Total params: 30,051,843
Trainable params: 30,051,843
Non-trainable params: 0
__________________________________________

In [47]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.5

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [48]:
text_ = generate_text(model, start_string=u"И вот идет уже ")
print(text_)

И вот идет уже знала.

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            


In [44]:
len(text_)

515